<a href="https://colab.research.google.com/github/ishancoderr/Rainfall_predictor/blob/main/Make_predictor_with_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the libraries**

In [133]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io
import requests

**Importing Rainfall data** 

In [134]:
url='https://raw.githubusercontent.com/ishancoderr/Rainfall_predictor/main/rainfall_data.csv'
s=requests.get(url).content
dataset=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [135]:
dataset.head()

,Unnamed: 0,Avg_Accumulation (mm),Duration (hr),Avg_intensity (mm/hr),occur_
0,0,257,6.0,43,1
1,1,268,5.0,53,1
2,2,289,6.0,48,1
3,3,215,4.0,54,1
4,4,149,3.0,50,1


In [136]:
dataset['occur_'].unique()

array([1, 0])

Hence occur_ can use as output value

In [137]:
dataset = dataset.to_numpy()

In [138]:
print(dataset.shape)

(50, 5)


In [139]:
print(dataset[0:5])

[[  0. 257.   6.  43.   1.]
 [  1. 268.   5.  53.   1.]
 [  2. 289.   6.  48.   1.]
 [  3. 215.   4.  54.   1.]
 [  4. 149.   3.  50.   1.]]


In [140]:
X=dataset[:,1:-1]

In [141]:
Y=dataset[:,-1]

In [142]:
print(X.shape,Y.shape)

(50, 3) (50,)


In [143]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [144]:
model=Sequential()


model.add(Dense(1,input_dim=len(X[0,:]),activation='sigmoid'))

In here use sigmoid funstion for classification

In [145]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [159]:
model.fit(x=X,y=Y,epochs=512,verbose=1)

Epoch 1/512
2/2 [==============================] - 0s 4ms/step - loss: 0.2183 - accuracy: 0.9600
Epoch 2/512
2/2 [==============================] - 0s 4ms/step - loss: 0.2171 - accuracy: 0.9600
Epoch 3/512
2/2 [==============================] - 0s 4ms/step - loss: 0.2202 - accuracy: 0.9600
Epoch 4/512
2/2 [==============================] - 0s 3ms/step - loss: 0.2163 - accuracy: 0.9600
Epoch 5/512
2/2 [==============================] - 0s 3ms/step - loss: 0.2177 - accuracy: 0.9600
Epoch 6/512
2/2 [==============================] - 0s 3ms/step - loss: 0.2173 - accuracy: 0.9600
Epoch 7/512
2/2 [==============================] - 0s 3ms/step - loss: 0.2215 - accuracy: 0.9400
Epoch 8/512
2/2 [==============================] - 0s 3ms/step - loss: 0.2124 - accuracy: 0.9600
Epoch 9/512
2/2 [==============================] - 0s 3ms/step - loss: 0.2119 - accuracy: 0.9600
Epoch 10/512
2/2 [==============================] - 0s 4ms/step - loss: 0.2126 - accuracy: 0.9600
Epoch 11/512
2/2 [===========

In [160]:
print(Y[:40])

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [161]:
predictions=model.predict(X)

In [162]:
print(predictions[:40].T)

[[0.99643886 0.99927413 0.9995414  0.99040353 0.8093476  0.93520784
  0.9687028  0.97670496 0.9999364  0.9996057  0.99985313 0.99978566
  0.9514791  0.96084034 0.88600063 0.9662471  0.88600063 0.9998416
  0.90801406 0.7169762  0.91443247 0.34009546 0.7169762  0.23151177
  0.23132095 0.23316595 0.07850081 0.05245906 0.08969337 0.3567087
  0.23206225 0.02384642 0.18390274 0.07223126 0.07306433 0.07747844
  0.0776532  0.11728179 0.32379407 0.56344324]]


In [173]:
model.save('/content/predtest.h5')

In [174]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [178]:
import tensorflow as tf

tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpulgoik6i/assets


INFO:tensorflow:Assets written to: /tmp/tmpulgoik6i/assets


In [179]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

980

In [180]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [1 3]
Input Type: <class 'numpy.float32'>
Output Shape: [1 1]
Output Type: <class 'numpy.float32'>
